# Part 1 : Getting a list of jobs per ROME

From the ROM_ArboPrincipale.csv given by Pole Emploie, we'll make an other file storing all the jobs linked to a ROME.  

In [1]:
# -- coding: utf-8 --
import pandas as pd 

ROME_arbo = pd.read_csv("DB/PE/ROME_ArboPrincipale.csv", encoding='UTF-8', error_bad_lines=False, sep=";")

In [2]:
ROME_arbo

,Lettre,deuxD,unD,nom,Code OGR
0,A,,,"Agriculture et Pêche, Espaces naturels et Espa...",
1,A,11,,Engins agricoles et forestiers,
2,A,11,01,Conduite d'engins agricoles et forestiers,
3,A,11,01,Chauffeur / Chauffeuse de machines agricoles,11987
4,A,11,01,Conducteur / Conductrice d'abatteuses,12862
...,...,...,...,...,...
11722,N,44,03,Opérateur / Opératrice de branchement du résea...,17064
11723,N,44,03,Opérateur / Opératrice de manoeuvre du réseau ...,17119
11724,N,44,03,Opérateur / Opératrice de production du réseau...,17150
11725,N,44,03,Opérateur / Opératrice fret du réseau ferré,17231


Here, we'll need only the rows with an OGR code. These are the jobs. 

In [3]:
metiers = ROME_arbo[ROME_arbo['Code OGR'] != " "]
metiers

,Lettre,deuxD,unD,nom,Code OGR
3,A,11,01,Chauffeur / Chauffeuse de machines agricoles,11987
4,A,11,01,Conducteur / Conductrice d'abatteuses,12862
5,A,11,01,Conducteur / Conductrice d'automoteur de récolte,38874
6,A,11,01,Conducteur / Conductrice d'engins d'exploitati...,13254
7,A,11,01,Conducteur / Conductrice d'engins d'exploitati...,13255
...,...,...,...,...,...
11722,N,44,03,Opérateur / Opératrice de branchement du résea...,17064
11723,N,44,03,Opérateur / Opératrice de manoeuvre du réseau ...,17119
11724,N,44,03,Opérateur / Opératrice de production du réseau...,17150
11725,N,44,03,Opérateur / Opératrice fret du réseau ferré,17231


We'll add a column called ROME wich is the ROME code of a job. We'll only keep ROME and nom in the new dataframe.

In [4]:
metiers['ROME'] = metiers['Lettre'] + metiers['deuxD'] + metiers['unD']

<ipython-input-4-54fdbc1b6db3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metiers['ROME'] = metiers['Lettre'] + metiers['deuxD'] + metiers['unD']


In [5]:
metiers = metiers[['ROME','nom']]
metiers

,ROME,nom
3,A1101,Chauffeur / Chauffeuse de machines agricoles
4,A1101,Conducteur / Conductrice d'abatteuses
5,A1101,Conducteur / Conductrice d'automoteur de récolte
6,A1101,Conducteur / Conductrice d'engins d'exploitati...
7,A1101,Conducteur / Conductrice d'engins d'exploitati...
...,...,...
11722,N4403,Opérateur / Opératrice de branchement du résea...
11723,N4403,Opérateur / Opératrice de manoeuvre du réseau ...
11724,N4403,Opérateur / Opératrice de production du réseau...
11725,N4403,Opérateur / Opératrice fret du réseau ferré


Now, we want to group our rows with the same ROME together. 

In [9]:
metiers.groupby('ROME')['nom'].apply(list).reset_index()

,ROME,nom
0,A1101,"[Chauffeur / Chauffeuse de machines agricoles,..."
1,A1201,"[Arboriste, Arboriste grimpeur / grimpeuse, Bû..."
2,A1202,[Agent / Agente d'aménagement des circuits péd...
3,A1203,[Agent / Agente d'entretien des parcs et jardi...
4,A1204,"[Agent / Agente technique des parcs nationaux,..."
...,...,...
527,N4301,"[Agent / Agente de conduite du réseau ferré, C..."
528,N4302,[Agent / Agente de contrôle des transports de ...
529,N4401,[Agent / Agente de circulation du réseau ferro...
530,N4402,[Agent / Agente d'exploitation des remontées m...


We save the dataframe into a CSV (but it will be saved without the groupby).

In [10]:
metiers.to_csv('ROME_jobs.csv',encoding='utf-8', index=False, sep=';')

Now that we have what we need. We'll use it 1) : to store the ROMEs linked to a job AND 2) scrapp all skills from a ROME

# Part 2: Recommendation of skills

## Getting top 5 skills from a Job

### Get occurences of skills in a categorie (AXX)

Given a job, we should be able to return the top 5 skills in the categorie of the job. 
Let's load the data.

In [11]:
import json
    
with open('DB/PE/PE_BDD.json', encoding = 'utf-8') as f:
    myDict_jobs = json.load(f)

Frist, from the name of a job, we'll get a ROME code then the first 3 char of the code (the categorie)

In [12]:
def getCategorie(rome):
    return rome[:3]

def getRome(job):
    for idex, row in metiers.iterrows():
        if job in row['nom']:
            return getCategorie(row['ROME'])

After that, we'll compute the occurence of a skill in a categorie. In order to do that, we'll use our dictionnary containing all the skills per ROME sheet. 

In [13]:
def occCategorie(job = '', categorieROME = ''):
    if(job != ''):
        categorieROME = getCategorie(getRome(job))
    skillsOccCategorie = {}
    for dict in myDict_jobs:
        if categorieROME in dict['ROME']:
            for skill in dict['skills']:
                if skill in skillsOccCategorie:
                    skillsOccCategorie[skill] += 1
                else:
                    skillsOccCategorie[skill] = 1
    return skillsOccCategorie

### Top 5 of skills

Now that we have a dictionnary with the most used skills in a categorie, we can sort it and get the top 5 (or top x) of the skills ! 

In [14]:
import operator

def topSkills(job='', categorie = '', nb=5):
    d = {}
    if job != '':
        d = occCategorie(job = job)
    else:
        d = occCategorie(categorieROME = categorie)
    s = sorted(d.items(), key=operator.itemgetter(1), reverse=True)
    s = s[:nb]
    return s

In [15]:
topSkills(job = 'Vendeur / Vendeuse en informatique')

[('Gestion comptable', 9),
 ('Gestion administrative', 9),
 ('Principes de la relation client', 9),
 ('Argumentation commerciale', 8),
 ('Techniques de vente', 8)]

## Getting top 5 skills of the "best" categorie of a skill

Now, given a skill, we'll check the occurences in all the categories. 
Once we've got the categorie where the skills appears the most, we'll get the top 5 of the skills from this categorie.

Of course, we'll re use topSkills method. But first, we need to get this categorie !

In [16]:
def occSkills(skill):
    skillsOccCategorie = {}
    for dict in myDict_jobs:
        if skill in dict['skills']:
            categorie = dict['ROME'][:3]
            if categorie in skillsOccCategorie:
                skillsOccCategorie[categorie] += 1
            else:
                skillsOccCategorie[categorie] = 1
    return skillsOccCategorie

In [17]:
def topSkillsFromSkill(skill):
    d= occSkills(skill)
    s = sorted(d.items(), key=operator.itemgetter(1), reverse=True)
    return topSkills(categorie = s[0][0])

In [18]:
topSkillsFromSkill("Droit du travail")

[('Droit civil', 3),
 ('Droit commercial', 3),
 ('Droit du travail', 3),
 ('Droit notarial', 2),
 ('Droit européen', 2)]